# 01 Einführung & Daten
------------------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Infos zum Datensatz

In [2]:
chunksize = 1_000_000
reader = pd.read_csv("../data/2023_Yellow_Taxi_Trip_Data.csv", chunksize=chunksize)

# Den ersten Chunk holen
first_chunk = next(reader)

# Info anzeigen
print(first_chunk.info())

FileNotFoundError: [Errno 2] No such file or directory: '../data/2023_Yellow_Taxi_Trip_Data.csv'

Wie wir sehen können ist der Datensatz recht gross und braucht viel Speicher, weshalb es sich nur in chunks bearbeiten lässt. 

Im Folgenden wird es vorallem darum gehen den Datensatz zu optimieren und wichtige Kennzahlen zu berechnen.

## Beschreiben des Datensatzes

In [ ]:
second_chunk = next(reader)

print(second_chunk.describe())

             VendorID  passenger_count   trip_distance      RatecodeID  \
count  1000000.000000   1000000.000000  1000000.000000  1000000.000000   
mean         1.732321         1.364828        3.372839        1.435825   
std          0.442750         0.896679       24.952718        6.013780   
min          1.000000         0.000000        0.000000        1.000000   
25%          1.000000         1.000000        1.060000        1.000000   
50%          2.000000         1.000000        1.760000        1.000000   
75%          2.000000         1.000000        3.250000        1.000000   
max          2.000000         9.000000     9683.760000       99.000000   

         PULocationID    DOLocationID    payment_type     fare_amount  \
count  1000000.000000  1000000.000000  1000000.000000  1000000.000000   
mean       166.420072      164.483686        1.215696       18.193923   
std         64.097900       69.799382        0.496670       17.590646   
min          1.000000        1.000000    

In [ ]:
df_header = pd.read_csv("../data/2023_Yellow_Taxi_Trip_Data.csv", nrows=0)
print("Der Datensatz enthält folgende Informationen")
original_header = df_header.columns.tolist()
print("Spaltennamen:\n", original_header)
print("\nAnzahl Header: ", len(original_header))

with open("../data/2023_Yellow_Taxi_Trip_Data.csv", "r", encoding="utf-8") as f:
    total_lines = sum(1 for _ in f) - 1

print(f"\nDie Datei hat ca. {total_lines:,} Zeilen.")

Der Datensatz enthält folgende Informationen
Spaltennamen:
 ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee']

Anzahl Header:  19

Die Datei hat ca. 38,310,226 Zeilen.


Um die Anzahl Zeilen zu erhalten konnte ich keine pandas einsetzen, da jedesmal das kernel kaputt gegangen ist. 
Unser Datensatz hat also um die 38 Millionen Zeilen/Taxifahrten im Jahr 2023 und zu einer Fahrt sind viel Daten vorhanden, wie zum Beispiel Preis, Uhrzeit und Distanz.

## Mögliche Datenverbesserung

Dank der Info- und Describe-Funktionen können wir uns nun den gesamten Datensatz ansehen und erkennen, welche Spalten für unsere Analyse nicht nützlich sind. Diese Spalten werden gelöscht, um Arbeitsspeicher (RAM) zu sparen und die Datengröße zu reduzieren.

- VendorID: Hat nur die Werte 1, 2 oder 6. Da wir diese Information nicht mit etwas Nützlichem verknüpfen können, kann die Spalte
            gelöscht werden.

- RatecodeID: Gleiches wie zuvor – die Werte sind meist 1 oder 99, in den meisten Fällen jedoch 1. Daher ist sie nicht hilfreich.

- store_and_fwd_flag: Immer derselbe Wert und die Bedeutung ist unklar. Diese Spalte kann ebenfalls entfernt werden.

- mta_tax: Hat immer denselben Wert (0.5) – es gibt zwar einige fehlerhafte Einträge, aber grundsätzlich keine Variation, daher kann
           sie gelöscht werden.

## Überprüfen der Spaltenlöschung

In [ ]:
print("Datensatz nach Löschung:\n")
df = pd.read_csv("../data/Taxi_Data_improved.csv", nrows=0)
print(f"Datensatz hat jetzt {df.shape[1]} Spalten")

Datensatz nach Löschung:

Datensatz hat jetzt 15 Spalten


## Erstellen eines repräsentativen Datensatzes

Unser Datensatz hat aktuell mehr als 38 Millionen Taxifahrten, es ist unmöglich (auch mit pandas) diesen in einem Stück einzulesen und zu analysieren, desshalb haben wir uns entschieden einen kleineren repräsentativen Datensatz zu erstellen. Zuerst wird das original chunkweise geshuffelt und anschliessen aus jedem gemischten chunk eine Prozentsatz an Daten random herausgenommen und in einem neuen Datensatz gespeichert. Der neue Datensatz sollte 1 Millionen Taxifahrten beinhalten.

In [ ]:
print(f"Daten des originalen Datensatzes: {total_lines:,}")

with open("../data/Taxi_sample_1M.csv", "r", encoding="utf-8") as f:
    new_total_lines = sum(1 for _ in f) - 1
print(f"Daten des neuen Datensatzes: {new_total_lines:,}")

Daten des originalen Datensatzes: 38,310,226
Daten des neuen Datensatzes: 999,997


## Wechsel zu Datetime

Aktuell liegt die Abfahrts und Ankunfts Zeit in der US-Datums form vor. Um die Zeiten für Rechnungen benutzen zu können, müssen wir die Zeitangaben in Datetime vorleigen haben. deshalb wurde das pythonscript change_to_datetime erstellt welches diese Angaben umwandelt

In [ ]:
example_1 = pd.read_csv("../data/2023_Yellow_Taxi_Trip_Data.csv", nrows=3)
print("Zeitangabe vor Änderung:\n")
print(example_1.iloc[:, 1:3])

example_2 = pd.read_csv("../data/Taxi_final_1M.csv", nrows=3)
print("\nZeitangabe nach Änderung:\n")
print(example_2.iloc[:, :2])

Zeitangabe vor Änderung:

     tpep_pickup_datetime   tpep_dropoff_datetime
0  01/01/2023 12:32:10 AM  01/01/2023 12:40:36 AM
1  01/01/2023 12:55:08 AM  01/01/2023 01:01:27 AM
2  01/01/2023 12:25:04 AM  01/01/2023 12:37:49 AM

Zeitangabe nach Änderung:

  tpep_pickup_datetime tpep_dropoff_datetime
0  2023-02-06 18:31:28   2023-02-06 18:41:28
1  2023-01-13 12:22:41   2023-01-13 12:54:42
2  2023-01-24 12:53:51   2023-01-24 13:06:09


## Generieren von neuen Spalten

Neben dem Wechsel zu Datetime, wird mit dem gleichen Pythonscript die Taxifahrtdauer und die durchschnitts Geschwindigkeit berechnet und hinten als neue Spalten eingefühgt. Zudem wird die Spalte trip_distance in Kilometer umgerechnet.

In [ ]:
example_3 = pd.read_csv("../data/Taxi_final_1M.csv", nrows=6)
print("\nNeu eingefühgte Spalten:\n")
print(example_3.iloc[:,15:])


Neu eingefühgte Spalten:

   trip_duration  average_speed
0           10.0           17.0
1           32.0            0.0
2           12.3           11.2
3            4.4           15.5
4           11.4           15.5
5            5.7           13.1
